<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#EDA" data-toc-modified-id="EDA-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>EDA</a></span><ul class="toc-item"><li><span><a href="#exploring-&quot;SCIENCE_DATA&quot;" data-toc-modified-id="exploring-&quot;SCIENCE_DATA&quot;-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>exploring "SCIENCE_DATA"</a></span><ul class="toc-item"><li><span><a href="#right,-i-forgot-there's-a-readme-explaining-everything-quite-clearly." data-toc-modified-id="right,-i-forgot-there's-a-readme-explaining-everything-quite-clearly.-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>right, i forgot there's a readme explaining everything quite clearly.</a></span></li><li><span><a href="#so-how-do-i-interpret-the-320x36?" data-toc-modified-id="so-how-do-i-interpret-the-320x36?-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>so how do i interpret the 320x36?</a></span><ul class="toc-item"><li><span><a href="#before-i-forget,-there's-probably-a-library-to-parse-lat/long-into-geographic-regions/cities/countries" data-toc-modified-id="before-i-forget,-there's-probably-a-library-to-parse-lat/long-into-geographic-regions/cities/countries-1.1.2.1"><span class="toc-item-num">1.1.2.1&nbsp;&nbsp;</span>before i forget, there's probably a library to parse lat/long into geographic regions/cities/countries</a></span></li></ul></li></ul></li><li><span><a href="#Data-format-is-based-off-of-instrument-format" data-toc-modified-id="Data-format-is-based-off-of-instrument-format-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data format is based off of instrument format</a></span></li><li><span><a href="#filename-considerations" data-toc-modified-id="filename-considerations-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>filename considerations</a></span></li></ul></li></ul></div>

In [25]:
import h5py
import os

# EDA

In [ ]:
fo = open("foo.txt", "r+")
str = fo.read(10);
print "Read String is : ", str
# Close opend file
fo.close()

In [ ]:
fo.read()

In [23]:
fo = open(filename, 'r+', encoding = "ISO-8859-1")
st = fo.read(2)
print(st)

H


In [24]:
for i in range(50):
    s = fo.read(i)
    print(s)


D
F


 
    
  
     
       
 ÿÿÿÿÿÿÿ
ÿ     
ÿÿÿÿÿÿÿÿ  
      `    
          
       ¨    
   ,       
          £v
     è      TRE
E   ÿÿÿÿÿÿÿÿÿÿÿÿ
ÿÿÿÿ        0    
                   
                    
                     
                      
                       
                        
                         
                          
                           
                            
                             
                              
                               
                                
                                 
                                  
                                   
                                    
           HEAP    °       `       1N
      0      
   long_name       
             Latitude               
                                  
                        x      
(                            
                      deflate     
       

In [19]:
import h5py
filename = "many_outs.h5"

In [20]:
with h5py.File(filename, "r",) as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])

OSError: Unable to open file (truncated file: eof = 622592, sblock->base_addr = 0, stored_eof = 1213323)

okay, found the issue
- for downloading multiple .h5 files, nasa reccomends:
    - cat <url.txt> | tr -d '\r' | xargs -n 1 curl -LJO -n -c ~/.urs_cookies -b ~/.urs_cookies
    - but i was getting some access denied + use --output recommendations. didn't work
        - so i tried >> piping it to a file. i could have named it "data.h5", maybe? can you store h5s in other h5s?
        - i named it 'data.iso' on someone's SO solution
        - this went poorly. the data downloaded, but when trying to open it, it seemed super corrupted, even with specific encoding that allowed it to open
    - i went back to NASA's downloading instructions and tested the curl cmd with a smaller subset of .h5 files, which worked (i think i had some typos in the cmdline, or i was formatting incorrectly) so i applied it to the original 2012-2020 46gb of data. currently downloading all the .h5s to my home directory, so i'll just have to move them en masse and loop-open them into a dataframe in the project folder (in 5 hours when it downloads).

In [ ]:
with h5py.File(filename, "r",) as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])

In [ ]:
# import os
# for filename in os.listdir(os.getcwd()):
#    with open(os.path.join(os.cwd(), filename), 'r') as f: # open in readonly mode
#       # do your stuff

In [90]:
# lst = []
# for filename in os.listdir(os.getcwd()):
#     if filename.endswith('.h5'):
#         with h5py.File(filename, 'r') as f:
#             print('keys: %s' %f.keys())
#             a_group_key = list(f.keys())[0]
#             data = list(f[a_group_key])

In [37]:
f = h5py.File('one.h5', 'r')

In [39]:
f.keys()

<KeysViewHDF5 ['ANCILLARY_DATA', 'GEOLOCATION_DATA', 'SCIENCE_DATA', 'nTimes', 'nWavel2', 'nWavel3', 'nXtrack']>

In [55]:
for k in f.keys():
    print(k, type(f[k]), '\n') # remember to use the key on the dict obj to actually get the type of value

ANCILLARY_DATA <class 'h5py._hl.group.Group'> 

GEOLOCATION_DATA <class 'h5py._hl.group.Group'> 

SCIENCE_DATA <class 'h5py._hl.group.Group'> 

nTimes <class 'h5py._hl.dataset.Dataset'> 

nWavel2 <class 'h5py._hl.dataset.Dataset'> 

nWavel3 <class 'h5py._hl.dataset.Dataset'> 

nXtrack <class 'h5py._hl.dataset.Dataset'> 



- so the root h5py object is a dictionary... of sorts.
    - we've got Groups, which are like folders, and Datasets, which are like arrays (but different)

In [53]:
f['GEOLOCATION_DATA'].keys() # looks like location of Suomi satellite at moment of reading
# we're gonna want Latitude and Longitude to identify 'where' the reading is.
# maybe TimeUTC as well, that could save a lot of re-coding work for broad regional targeting

<KeysViewHDF5 ['Latitude', 'Longitude', 'SolarAzimuthAngle', 'SolarZenithAngle', 'SpacecraftAltitude', 'SpacecraftLatitude', 'SpacecraftLongitude', 'TimeTAI93', 'TimeUTC', 'ViewingAzimuthAngle', 'ViewingZenithAngle']>

In [56]:
f['ANCILLARY_DATA'].keys()

<KeysViewHDF5 ['CloudPressure', 'TerrainPressure']>

In [57]:
f['SCIENCE_DATA'].keys()

<KeysViewHDF5 ['CloudFraction', 'ColumnAmountO3', 'ColumnAmountSO2_PBL', 'ColumnAmountSO2_STL', 'ColumnAmountSO2_TRL', 'ColumnAmountSO2_TRM', 'ColumnAmountSO2_TRU', 'FittingWindow_STL', 'FittingWindow_TRL', 'FittingWindow_TRM', 'FittingWindow_TRU', 'Flag_SAA', 'Flag_SO2', 'RadiativeCloudFraction', 'Reflectivity331', 'SLER', 'UVAerosolIndex', 'Wavelengths_SLER', 'dNdR', 'nPrincipalComponents']>

from the NASA dataset description:

The goal of this dataset is to create and archive a Level 2 Sulfur Dioxide (SO2) Earth Science Data Record (ESDR) from backscatter Ultraviolet (BUV) measurements from the Ozone Mapping and Profiler Suite (OMPS) Nadir Mapper (NM) flown on the NASA/NOAA Suomi National Polar-orbiting Partnership (S-NPP) satellite since 2011. We apply the NASA Goddard Space Flight Center (GSFC) principal component analysis (PCA) satellite trace gas retrieval algorithm, to obtain the best measurement-based ESDR of volcanic and anthropogenic SO2 loadings and emissions, which is fully consistent with our SO2 ESDR (OMSO2) from the Ozone Monitoring Instrument (OMI) flown on the NASA Earth Observing System (EOS) Aura satellite since 2004. 

The SO2 ESDR released here at the Goddard Earth Science (GES) Data and Information Data Center (DISC), OMPS_NPP_NMSO2_PCA_L2, is a stand-alone dataset, but will also be part of the NASA's Making Earth System Data Records for Use in Research Environments (MEaSUREs) program. It covers the entire period of the S-NPP/OMPS mission since launch in October 2011, and forward data processing is ongoing. OMPS_NPP_NMSO2_PCA_L2 is a Level 2 orbital swath product, which will be used to study SO2 pollutants emitted from large point sources such as coal-fired power plants and smelters as well as to monitor volcanic SO2 emissions.

Sulfur Dioxide (SO2) is a short-lived gas primarily produced by volcanoes, power plants, refineries, metal smelting and burning of fossil fuels. Where SO2 remains near the Earth's surface, it is toxic, causes acid rain, and degrades air quality. Where SO2 is lofted into the free troposphere, it forms aerosols that can alter cloud reflectivity and precipitation. In the stratosphere, volcanic SO2 forms sulfate aerosols that can result in climate change.

- translation:
    - here's SO2 levels by latitude, date & many other variables
        - derived from applying PCA algos to Ozone mapping data, which was measured from backscatter ultraviolet readings
                - basically, NASA looks at backscattered UV off of the atmosphere - off of ozone, but can effectively derive SO2 from the ozone reading using PCA
        - seems convoluted, but it's consistent with their last SO2 reading system from 2004, and it's probably the best estimate the world's got right now (unless anyone would like to step up and accurately measure trace gas concentrations globally every day. seems bloody hard in my opinion)

## exploring "SCIENCE_DATA"

In [63]:
f['SCIENCE_DATA'].keys()

<KeysViewHDF5 ['CloudFraction', 'ColumnAmountO3', 'ColumnAmountSO2_PBL', 'ColumnAmountSO2_STL', 'ColumnAmountSO2_TRL', 'ColumnAmountSO2_TRM', 'ColumnAmountSO2_TRU', 'FittingWindow_STL', 'FittingWindow_TRL', 'FittingWindow_TRM', 'FittingWindow_TRU', 'Flag_SAA', 'Flag_SO2', 'RadiativeCloudFraction', 'Reflectivity331', 'SLER', 'UVAerosolIndex', 'Wavelengths_SLER', 'dNdR', 'nPrincipalComponents']>

In [67]:
f['SCIENCE_DATA']['ColumnAmountSO2_PBL']

<HDF5 dataset "ColumnAmountSO2_PBL": shape (320, 36), type "<f4">

In [68]:
pbl = f['SCIENCE_DATA']['ColumnAmountSO2_PBL']

In [77]:
def peek_dataset(ds): # only works on 2d array datasets
    for i in range(ds.shape[0]):
        for k in range(ds.shape[1]):
            print(ds[i][k])
        print('\n')

In [82]:
# peek_dataset(pbl) # a 320x36 matrix. now to figure out why there's 11520 readings in this dataset, which is one .h5 file


In [89]:
for k in f['SCIENCE_DATA'].keys(): # are they all 320x36? probably the SO2 at least
#     print(k, type(k))
    if 'SO2' in k: # k is a string
        print(k, f['SCIENCE_DATA'][k].shape) # yes, the SO2 readings are matched in size.
# probably different instruments or estimation methods. perhaps we could aggregate them later on, but they're all going in the dataframe for now

ColumnAmountSO2_PBL (320, 36)
ColumnAmountSO2_STL (320, 36)
ColumnAmountSO2_TRL (320, 36)
ColumnAmountSO2_TRM (320, 36)
ColumnAmountSO2_TRU (320, 36)
Flag_SO2 (320, 36)


### right, i forgot there's a readme explaining everything quite clearly.
- units
    - 5 estimates (principal components), related to various geophysical processes (ozone absorption, rotational Raman scattering) and instrument measurement details (wavelength shift, dark current)
    - PCs used in SO2 spectral fitting to reduce interferences
        - so we've got five estimates of total "SO2 Vertical Column Density" in Dobson Units (1DU = 2.69 * 10^16 molecules/cm^2)
            - note the "cm^2". these are flat "slices" of a column? no
            - each of the 5 estimates corresponds to a different assumed SO2 cloud height, or Center of Mass Altitude (CMA)
            - all 5 VCD values represent the estimated total SO2 burden within entire atmospheric column:
                - should not be interpreted as partial column amounts within different layers/parts of atmosphere
                - they want users to select 1 or interpolate between two estimates that are most representative of conditions for a particular case of interest
- so each of the 5 estimates measures with different heights/sensitivities, and therefore are 'tuned' to measure different... types of SO2 emission.
    - anthropogenic (coal factories etc) SO2 emissions will look different, and be measured differently from volcanic eruptions
    - that sort of thing

- PBL: Planetary Boundary Layer
    - assumes that SO2 is predominantly in lowest 1km of atmosphere
    - use for studies on near-surface pollution
        - for each OMPS orbit:
            - process 36 rows (cross-track positions) one at a time
            - use PCA to extract PCs for spectral range 310.5-340nm from sun-normalized BUV radiance spectra
            - PCs are ranked in descending order according to spectral variance that they explain
    - due to jacobian simplication, they recommend only using snow/ice free pixels with small radiative cloud fraction
- the other 4 column amounts seem to be volcanic
### so how do i interpret the 320x36?
- wait, they're talking about pixels
    - 320x36 is the "resolution" of the measurement
    
#### before i forget, there's probably a library to parse lat/long into geographic regions/cities/countries

- it's an "orbital-track" product about the SNPP satellite.
    - polar sun-synchronous orbit
## Data format is based off of instrument format
- OMPS-NM has 110° FoV, cross-track swath ~= 2800km
    - global coverage,14-15 orbits/day
    - "nominal spatial resolution" is 50km x 50km at nadir in nominal observation mode
        - there's a higher res mode recorded once a week, but that's not in this dataset
- for each OMPS orbit, process 36 rows (cross-track positions) use PCA to extract PCs for spectral range from BUV
    - so, 320 PCs per position while crossing orbital track?
        - what does the lat/long data look like? that might solve this question


In [ ]:
lat = f['GEOLOCATION_DATA']['Latitude']
long = f['GEOLOCATION_DATA']['Longitude']
print(lat.shape, long.shape)

- so in one .h5's 36x320:
    - began at -29, -152: about 1/3 of the way from NZ to Ecuador
    - ended at 67, 16: northern middle Norway
        - but those aren't the extreme bounds for lat or long.
            - without plotting it out, my intuition says it's an oscillating path

In [98]:
# peek_dataset(lat)
# peek_dataset(long)

- now the 50km spatial resolution makes sense.
    - we've got 11520 'pixels' or 'granules' (are granules the .h5?)
    - each pixel has an SO2 PBL reading, as well as a lat/long
        - so the corresponding [x],[y] position in the matrix will match a geolocation to its PBL
- warnings from the readme:
    - all pixels:
        - with large solar zenith angle (SZA>70)
        - near edge of swath (rows 1-2, 35-36)
        - affected by South Atlantic Anomaly (flag_SAA=1)
            - should be excluded
    - PBL SO2:
        - only use:
            - snow/ice free pixels
            - with RadiativeCloudFraction (<0.3)
            
    

## filename considerations
- each .h5 file contains:
    - date/time at start of orbit
    - orbit number
        - so each .h5 is an orbit. question answered
- what about time? that should help me figure out "unique positions"

In [109]:
geo = f['GEOLOCATION_DATA']
time = geo['TimeUTC']
time.shape # ah

(320,)

- so we've got 320 times per orbit, but 320x36 pixels and 320x36 lat/long coords
- that means at 320 times throughout the day, the satellite gathers 36 pixels, and each one has a unique geolocation...?
    - the 36 is "cross-track positions"

## right, i think i've got the format
- each .h5 file is an orbit, with 36

(320,)

In [64]:
f['GEOLOCATION_DATA'].keys()

<KeysViewHDF5 ['Latitude', 'Longitude', 'SolarAzimuthAngle', 'SolarZenithAngle', 'SpacecraftAltitude', 'SpacecraftLatitude', 'SpacecraftLongitude', 'TimeTAI93', 'TimeUTC', 'ViewingAzimuthAngle', 'ViewingZenithAngle']>

In [92]:
lat = f['GEOLOCATION_DATA']['Latitude']
lat.shape

(320, 36)

In [62]:
f['ANCILLARY_DATA'].keys()

<KeysViewHDF5 ['CloudPressure', 'TerrainPressure']>

In [51]:
nt = f['nTimes']
print(nt.shape, nt.dtype) 

(320,) int32


- this "nTimes" key has 320 ints lined up 1D